Temporal Innovator 

In [ ]:
#%pip install mesa
#pip install ipykernel
#pip install ipywidgets --upgrade

In [3]:
from mesa import Agent, Model
from mesa.space import MultiGrid
import random

In [9]:
class TimeAgent(Agent):
    def __init__(self, model):
        super().__init__(model)
        # self.pos will be set by place_agent()

    def step(self):
        # Move randomly: up, down, left, right
        x, y = self.pos  # Now safe after place_agent()
        move = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])
        new_pos = (x + move[0], y + move[1])
        self.model.grid.move_agent(self, new_pos)

class TimeModel(Model):
    def __init__(self):
        super().__init__()
        self.grid = MultiGrid(10, 10, True)
        self.schedule = []  # Manual agent list
        self.random = random.Random()
        for i in range(3):  # 3 agents
            agent = TimeAgent(self)
            self.grid.place_agent(agent, (0, 0))  # Explicitly place at (0, 0)
            self.schedule.append(agent)

    def step(self):
        random.shuffle(self.schedule)  # Random activation
        for agent in self.schedule:
            agent.step()

# Run the model
model = TimeModel()
for _ in range(5):  # Run 5 steps
    model.step()